# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("../WeatherPy/Output_cities.csv")
cities_df["max_temp"]=cities_df["max_temp"]-273.15
cities_df["max_temp"].max()


34.150000000000034

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
 # Create a poverty Heatmap layer
fig = gmaps.figure()

locations = cities_df[["lat","lon"]].astype(float)
humidity = cities_df["humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cities_df = cities_df[(cities_df["max_temp"] < 30) & (cities_df["max_temp"] > 28)]
cities_df

,city_name,city_number,cloudiness,country,date,humidity,lat,lon,max_temp,win_peed
39,Pisco,3932145,0,PE,1613163098,54,-13.7000,-76.2167,29.00,7.72
67,San Juan Bautista,3929493,0,PE,1613163300,54,-14.0108,-75.7358,29.00,7.72
77,Pitimbu,3391889,75,BR,1613163103,70,-7.4706,-34.8086,29.00,6.69
133,Touros,3386213,40,BR,1613163101,70,-5.1989,-35.4608,29.00,7.20
167,Lufilufi,4035249,38,WS,1613163318,70,-13.8745,-171.5986,28.32,3.73
201,Yuty,3436547,48,PY,1613163323,69,-26.6000,-56.2500,28.07,1.65
236,Tual,1623197,100,ID,1613163101,79,-5.6667,132.7500,28.61,4.78
285,Carauari,3664659,86,BR,1613163146,81,-4.8828,-66.8958,28.57,0.94
296,Autazes,3406948,95,BR,1613163339,77,-3.5797,-59.1306,28.95,0.73
380,Cabedelo,3404558,75,BR,1613163354,70,-6.9811,-34.8339,29.00,6.69


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = cities_df.head(3)
hotel_df["Hotel Name"] =""


for index, row in hotel_df.iterrows():
    
    target_coordinates = f'{row["lat"]},{row["lon"]}'
    target_search = ""
    target_radius = 5000
    target_type = "hotel"
    
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    rep = response.json()
    hotel_df.at[index,"Hotel Name"] = rep["results"][1]["name"]
    pprint(rep["results"][1]["name"])
    

<ipython-input-35-540189f5c0c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] =""


'Hotel San Isidro Oficial'
'Hotel Las Dunas'
'City of Pitimbu'


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
locations = hotel_df[["lat","lon"]].astype(float)
hotel_names = hotel_df["Hotel Name"]
# Display figure
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2
    
)
fig = gmaps.figure()
fig.add_layer(hotel_layer)

,city_name,city_number,cloudiness,country,date,humidity,lat,lon,max_temp,win_peed,Hotel Name
39,Pisco,3932145,0,PE,1613163098,54,-13.7000,-76.2167,29.0,7.72,Hotel San Isidro Oficial
67,San Juan Bautista,3929493,0,PE,1613163300,54,-14.0108,-75.7358,29.0,7.72,Hotel Las Dunas
77,Pitimbu,3391889,75,BR,1613163103,70,-7.4706,-34.8086,29.0,6.69,City of Pitimbu
